In [ ]:
# bring in our LLAMA_CLOUD_API_KEY

import nest_asyncio
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from openai import AzureOpenAI
from typing import Optional, Dict
from smolagents.models import OpenAIServerModel
from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool


from prompt_library import decoding_manual_prompt, logical_questions_prompt, logical_answer_prompt,agent_prompt,enhancing_vessel_details_prompt


nest_asyncio.apply()
def parse_document(file_name, llama_cloud_api_key):
    
    # bring in deps
    from llama_cloud_services import LlamaParse
    from llama_index.core import SimpleDirectoryReader

    # set up parser
    parser = LlamaParse(
        api_key=llama_cloud_api_key,
        result_type="markdown"  # "markdown" and "text" are available
    )

    # use SimpleDirectoryReader to parse our file
    file_extractor = {".pdf": parser}
    documents = SimpleDirectoryReader(input_files=[file_name], file_extractor=file_extractor).load_data()
    contents=[documents[int(id)].text_resource.text for id in range(2,len(documents))]
    return contents


def creating_index(contents):
    # Step 1: Load documents
    documents = contents

    
    model_embed = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model_embed.encode(documents, convert_to_numpy=True)

    # Step 3: Normalize embeddings for cosine similarity
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    # Step 4: Use FAISS inner product index (cosine similarity)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings)
    faiss.write_index(index, "my_faiss_index.index")
    
    
    
def search_manual_index(query):

    index = faiss.read_index("my_faiss_index.index")
    model_embed = SentenceTransformer('all-MiniLM-L6-v2')
    # Step 5: Normalize the query and search
    query =query
    query_embedding = model_embed.encode([query], convert_to_numpy=True)
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

    D, I = index.search(query_embedding, k=1)
    return I[0].item()

def decode_manual(client,prompt,model,manual):
    prompt=prompt.format(manual=manual)
    
    # print(prompt)
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": prompt}
        ]
    )
    instructions=response.choices[0].message.content
    return instructions

def logical_questions_generation(client,prompt,model,manual):
    prompt=prompt.format(manual=manual)
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": prompt}
        ]
    )
    questions=response.choices[0].message.content
    return questions
    

def logical_answers_generation(client,prompt,model,vessel_details, questions):
    prompt=prompt.format(questions=questions,vessel_details=vessel_details)
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": prompt}
        ]
    )
    questions=response.choices[0].message.content
    return questions

def initiate_openai_model(openai_api_key,api_version,azure_endpoint):
    client = AzureOpenAI(
    azure_endpoint = azure_endpoint,
    api_key=openai_api_key ,
    api_version=api_version
    )
    return client

def tariff_calculator_agent(model_id,api_key,api_version,azure_endpoint, agent_prompt, vessel_details,instructions,answer):
    
    class AzureOpenAIServerModel(OpenAIServerModel):
        """This model connects to an Azure OpenAI deployment.

        Parameters:
            model_id (`str`):
                The model identifier to use on the server (e.g. "gpt-3.5-turbo").
            azure_endpoint (`str`, *optional*):
                The Azure endpoint, including the resource, e.g. `https://example-resource.azure.openai.com/`
            api_key (`str`, *optional*):
                The API key to use for authentication.
            custom_role_conversions (`Dict{str, str]`, *optional*):
                Custom role conversion mapping to convert message roles in others.
                Useful for specific models that do not support specific message roles like "system".
            **kwargs:
                Additional keyword arguments to pass to the Azure OpenAI API.
        """

        def __init__(
            self,
            model_id: str,
            azure_endpoint: Optional[str] = None,
            api_key: Optional[str] = None,
            api_version: Optional[str] = None,
            custom_role_conversions: Optional[Dict[str, str]] = None,
            **kwargs,
        ):
            super().__init__(model_id=model_id, api_key=api_key, custom_role_conversions=custom_role_conversions, **kwargs)
            # if we've reached this point, it means the openai package is available (baseclass check) so go ahead and import it
            import openai

            self.client = openai.AzureOpenAI(
                api_key=api_key,
                api_version=api_version,
                azure_endpoint=azure_endpoint
            )
            
    model = AzureOpenAIServerModel(
    model_id = model_id,
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint
    )
    
    agent = CodeAgent(tools=[], model=model)
    agent_prompt=agent_prompt.format(vessel_details=vessel_details,instructions= instructions, answer=answer)
    ans=agent.run(agent_prompt)
    return ans

def enhancing_vessel_info(client,model,questions,vessel_details,prompt):
    prompt=prompt.format(questions=questions,vessel_details=vessel_details)
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": prompt}
        ]
    )
    enhanced_vessel_details=response.choices[0].message.content
    return enhanced_vessel_details
    
def main():
    
    ##### Pass the necessary inputs here ##########################################
    
    are_you_running_code_first_time=True # If True, it will creta eand save vector index, if False, then it will use already created index.
    file_path =""
    llama_cloud_api_key=""
    openai_api_key=""
    azure_endpoint = ""
    api_version=""
    model= ""

    vessel_details="""
    Calculate the different tariffs payable by the following vessel berthing at the port of Durban
    Vessel Details:

    General
    Vessel Name: SUDESTADA
    Built: 2010
    Flag: MLT - Malta
    Classification Society: Registro Italiano Navale
    Call Sign: [Not provided]

    Main Details
    Lloyds / IMO No.: [Not provided]
    Type: Bulk Carrier
    DWT: 93,274
    GT / NT: 51,300 / 31,192
    LOA (m): 229.2
    Beam (m): 38
    Moulded Depth (m): 20.7
    LBP: 222
    Drafts SW S / W / T (m): 14.9 / 0 / 0
    Suez GT / NT: - / 49,069

    Communication
    E-mail: [Not provided]
    Commercial E-mail: [Not provided]
    DRY
    Number of Holds: 7
    
    Cargo Details
    Cargo Quantity: 40,000 MT
    Days Alongside: 3.39 days
    Arrival Time: 15 Nov 2024 10:12
    Departure Time: 22 Nov 2024 13:00

    Activity/Operations
    Activity: Exporting Iron Ore
    Number of Operations: 2
    """
    
    list_of_dues=[
    "light dues",
    "port dues",
    "towage dues",
    "vehicle traffic services (VTS) dues",
    "pilotage dues",
    "running of vessel lines dues"
    ]
    
    ######################### Input ends here #######################################
    
    
    final_result={}
    contents=parse_document(file_path,llama_cloud_api_key)
    
    for i in list_of_dues:
        print("Calculating "+i)
        Query = "I need to calculate "+i+" for my vessel."
        if are_you_running_code_first_time:
            creating_index(contents)
            id=search_manual_index(Query)
        else:
            id=search_manual_index(Query)
            
        manual=contents[id]
        # print(manual)
        print("--------------------------------")
        openai_client=initiate_openai_model(openai_api_key,api_version,azure_endpoint)
        
        manual_elaboration= decode_manual(openai_client,decoding_manual_prompt, model,manual)
        
        questions = logical_questions_generation(openai_client, logical_questions_prompt, model, manual )
        # print(questions)
        
        enhanced_vessel_details = enhancing_vessel_info(openai_client,model,questions,vessel_details,enhancing_vessel_details_prompt)
        print(enhanced_vessel_details)
        answers= logical_answers_generation(openai_client, logical_answer_prompt, model,enhanced_vessel_details,questions)
        # print(answers)
        
        final_answer=tariff_calculator_agent(model,openai_api_key,api_version,azure_endpoint, agent_prompt,enhanced_vessel_details,manual_elaboration,answers)
        
        final_result[i]=final_answer
        
    return final_result
        
    

In [ ]:
main()


In [ ]:
























    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
DeprecationWarning


















































    
    
    
    
    
    
    
    












